In [ ]:
import json
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
import boto3
from nhl_pipeline.config import get_settings

settings = get_settings()
print(f"S3 Bucket: {settings.s3_bucket}")

In [ ]:
# Initialize S3 client
s3 = boto3.client('s3', region_name=settings.aws_region)
bucket = settings.s3_bucket

## 1. Load Sample Odds Data

In [ ]:
# List available odds dates
odds_prefix = "raw/odds/player_props/market=player_shots_on_goal/"
response = s3.list_objects_v2(Bucket=bucket, Prefix=odds_prefix, Delimiter='/')

odds_dates = []
for prefix in response.get('CommonPrefixes', []):
    date = prefix['Prefix'].split('date=')[1].rstrip('/')
    odds_dates.append(date)

odds_dates = sorted(odds_dates)
print(f"Odds data available for {len(odds_dates)} dates")
print(f"Date range: {odds_dates[0] if odds_dates else 'N/A'} to {odds_dates[-1] if odds_dates else 'N/A'}")
print(f"\nRecent dates: {odds_dates[-5:] if len(odds_dates) >= 5 else odds_dates}")

In [ ]:
def load_odds_for_date(date: str) -> list[dict]:
    """Load all odds events for a specific date."""
    prefix = f"raw/odds/player_props/market=player_shots_on_goal/date={date}/"
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    
    events = []
    for obj in response.get('Contents', []):
        resp = s3.get_object(Bucket=bucket, Key=obj['Key'])
        data = json.loads(resp['Body'].read().decode('utf-8'))
        events.append(data)
    
    return events

# Load most recent date
if odds_dates:
    sample_date = odds_dates[-1]
    print(f"Loading odds for {sample_date}...")
    sample_events = load_odds_for_date(sample_date)
    print(f"Loaded {len(sample_events)} events")

In [ ]:
# Parse player props from one event
def extract_player_props(event: dict) -> pd.DataFrame:
    """Extract player props from an odds event."""
    rows = []
    
    game_date = event.get('game_date')
    event_id = event.get('event_id')
    home_team = event.get('home_team')
    away_team = event.get('away_team')
    
    data = event.get('data', {})
    bookmakers = data.get('bookmakers', [])
    
    for bm in bookmakers:
        bookmaker = bm.get('key')
        for market in bm.get('markets', []):
            for outcome in market.get('outcomes', []):
                rows.append({
                    'game_date': game_date,
                    'event_id': event_id,
                    'home_team': home_team,
                    'away_team': away_team,
                    'bookmaker': bookmaker,
                    'player_name': outcome.get('description'),
                    'bet_type': outcome.get('name'),  # Over/Under
                    'line': outcome.get('point'),
                    'odds': outcome.get('price'),
                })
    
    return pd.DataFrame(rows)

# Parse all events for sample date
if odds_dates:
    odds_dfs = [extract_player_props(e) for e in sample_events]
    odds_df = pd.concat(odds_dfs, ignore_index=True) if odds_dfs else pd.DataFrame()
    print(f"Parsed {len(odds_df)} prop lines")
    
    # Show unique players
    print(f"\nUnique players with props: {odds_df['player_name'].nunique()}")
    print("\nSample player names from odds:")
    print(odds_df['player_name'].drop_duplicates().head(20).tolist())

## 2. Load Actual Game Stats (SOG)

In [ ]:
def load_boxscore_for_date(date: str) -> list[dict]:
    """Load boxscore data for games on a specific date."""
    # Boxscores are stored under gamecenter with game_id
    prefix = "raw/nhl/gamecenter/"
    
    # We need to find games that match this date
    # Games are stored by ID, so we'll list and filter
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter='/')
    
    boxscores = []
    for prefix_obj in response.get('CommonPrefixes', []):
        game_prefix = prefix_obj['Prefix']
        # List files in this game folder
        game_resp = s3.list_objects_v2(Bucket=bucket, Prefix=game_prefix)
        for obj in game_resp.get('Contents', []):
            if 'boxscore' in obj['Key']:
                resp = s3.get_object(Bucket=bucket, Key=obj['Key'])
                data = json.loads(resp['Body'].read().decode('utf-8'))
                # Check if game date matches
                game_date = data.get('gameDate', '')
                if game_date == date:
                    boxscores.append(data)
    
    return boxscores

print("Note: Loading boxscores can be slow. Using a targeted approach...")

In [ ]:
# Alternative: Load boxscore by game_id from the schedule
def find_games_on_date(date: str) -> list[str]:
    """Find game IDs for a specific date from schedule snapshots."""
    schedule_prefix = f"raw/nhl/schedule/date={date}/"
    
    try:
        response = s3.list_objects_v2(Bucket=bucket, Prefix=schedule_prefix)
        if not response.get('Contents'):
            return []
        
        # Get the most recent snapshot
        latest_key = sorted([o['Key'] for o in response['Contents']])[-1]
        resp = s3.get_object(Bucket=bucket, Key=latest_key)
        data = json.loads(resp['Body'].read().decode('utf-8'))
        
        game_ids = []
        for week in data.get('gameWeek', []):
            if week.get('date') == date:
                for game in week.get('games', []):
                    game_ids.append(str(game['id']))
        
        return game_ids
    except Exception as e:
        print(f"Error: {e}")
        return []

if odds_dates:
    game_ids = find_games_on_date(sample_date)
    print(f"Found {len(game_ids)} games on {sample_date}: {game_ids}")

In [ ]:
def load_boxscore(game_id: str) -> dict | None:
    """Load boxscore for a specific game."""
    prefix = f"raw/nhl/gamecenter/game_id={game_id}/"
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    
    for obj in response.get('Contents', []):
        if 'boxscore' in obj['Key']:
            resp = s3.get_object(Bucket=bucket, Key=obj['Key'])
            return json.loads(resp['Body'].read().decode('utf-8'))
    return None

def extract_player_sog(boxscore: dict) -> pd.DataFrame:
    """Extract player shots on goal from boxscore."""
    rows = []
    
    game_id = boxscore.get('id')
    game_date = boxscore.get('gameDate')
    
    player_stats = boxscore.get('playerByGameStats', {})
    
    for team_key in ['homeTeam', 'awayTeam']:
        team_data = player_stats.get(team_key, {})
        team_name = boxscore.get(team_key, {}).get('name', {}).get('default', 'Unknown')
        team_abbrev = boxscore.get(team_key, {}).get('abbrev', '')
        
        for position in ['forwards', 'defense']:
            for player in team_data.get(position, []):
                sog = player.get('sog', 0)
                if sog is not None:
                    rows.append({
                        'game_id': game_id,
                        'game_date': game_date,
                        'player_id': player.get('playerId'),
                        'player_name': player.get('name', {}).get('default', ''),
                        'position': player.get('position'),
                        'team_name': team_name,
                        'team_abbrev': team_abbrev,
                        'home_away': team_key.replace('Team', ''),
                        'shots_on_goal': sog,
                    })
    
    return pd.DataFrame(rows)

# Load boxscores for sample date
if odds_dates and game_ids:
    all_sog = []
    for gid in game_ids:
        box = load_boxscore(gid)
        if box:
            all_sog.append(extract_player_sog(box))
    
    actuals_df = pd.concat(all_sog, ignore_index=True) if all_sog else pd.DataFrame()
    print(f"Loaded {len(actuals_df)} player stats")
    print("\nSample player names from NHL:")
    print(actuals_df['player_name'].drop_duplicates().head(20).tolist())

## 3. Player Name Matching Analysis

In [ ]:
# Get unique player names from both sources
if 'odds_df' in dir() and len(odds_df) > 0 and 'actuals_df' in dir() and len(actuals_df) > 0:
    odds_names = set(odds_df['player_name'].dropna().unique())
    nhl_names = set(actuals_df['player_name'].dropna().unique())
    
    print(f"Unique player names in odds data: {len(odds_names)}")
    print(f"Unique player names in NHL data: {len(nhl_names)}")
    
    # Exact matches
    exact_matches = odds_names & nhl_names
    print(f"\nExact matches: {len(exact_matches)} ({100*len(exact_matches)/len(odds_names):.1f}%)")
    
    # Unmatched odds names
    unmatched_odds = odds_names - nhl_names
    print(f"Unmatched odds names: {len(unmatched_odds)}")
    print("\nSample unmatched odds names:")
    for name in sorted(unmatched_odds)[:15]:
        print(f"  {name}")

In [ ]:
# Try fuzzy matching
from difflib import SequenceMatcher, get_close_matches

def find_best_match(odds_name: str, nhl_names: list[str], cutoff: float = 0.6) -> tuple[str, float] | None:
    """Find best matching NHL name for an odds name."""
    matches = get_close_matches(odds_name, nhl_names, n=1, cutoff=cutoff)
    if matches:
        ratio = SequenceMatcher(None, odds_name.lower(), matches[0].lower()).ratio()
        return matches[0], ratio
    return None

if 'odds_df' in dir() and len(odds_df) > 0 and 'actuals_df' in dir() and len(actuals_df) > 0:
    nhl_names_list = list(actuals_df['player_name'].dropna().unique())
    
    print("Fuzzy matching unmatched odds names:\n")
    match_results = []
    for odds_name in sorted(unmatched_odds)[:20]:
        result = find_best_match(odds_name, nhl_names_list)
        if result:
            nhl_name, score = result
            match_results.append({'odds_name': odds_name, 'nhl_name': nhl_name, 'score': score})
            print(f"  {odds_name:25} -> {nhl_name:25} ({score:.2f})")
        else:
            print(f"  {odds_name:25} -> NO MATCH")
    
    match_df = pd.DataFrame(match_results)

## 4. Build a Name Mapping Strategy

Common differences:
- Last name only vs full name
- Name abbreviations (J. vs Jake)
- Accented characters
- Suffix differences (Jr., III)

In [ ]:
import unicodedata
import re

def normalize_name(name: str) -> str:
    """Normalize a player name for matching."""
    if not name:
        return ''
    
    # Convert to lowercase
    name = name.lower().strip()
    
    # Remove accents
    name = unicodedata.normalize('NFD', name)
    name = ''.join(c for c in name if unicodedata.category(c) != 'Mn')
    
    # Remove suffixes
    name = re.sub(r'\s+(jr\.?|sr\.?|ii|iii|iv)$', '', name)
    
    # Remove extra whitespace
    name = ' '.join(name.split())
    
    return name

def get_last_name(name: str) -> str:
    """Extract last name from full name."""
    parts = name.strip().split()
    return parts[-1] if parts else ''

# Test normalization
test_names = [
    "Connor McDavid",
    "Léon Draisaitl",
    "Auston Matthews",
    "J.T. Miller",
    "T.J. Oshie"
]

print("Name normalization examples:")
for name in test_names:
    print(f"  {name:20} -> {normalize_name(name)}")

In [ ]:
def match_player_names(odds_name: str, nhl_names_df: pd.DataFrame) -> dict | None:
    """
    Match an odds player name to an NHL player.
    
    Matching strategies:
    1. Exact match (normalized)
    2. Last name match (if unique on that team/game)
    3. Fuzzy match
    """
    odds_norm = normalize_name(odds_name)
    odds_last = normalize_name(get_last_name(odds_name))
    
    # Strategy 1: Exact normalized match
    for _, row in nhl_names_df.iterrows():
        nhl_norm = normalize_name(row['player_name'])
        if odds_norm == nhl_norm:
            return {
                'nhl_player_name': row['player_name'],
                'player_id': row['player_id'],
                'match_type': 'exact',
                'confidence': 1.0
            }
    
    # Strategy 2: Last name match
    last_name_matches = []
    for _, row in nhl_names_df.iterrows():
        nhl_last = normalize_name(get_last_name(row['player_name']))
        if odds_last == nhl_last:
            last_name_matches.append(row)
    
    if len(last_name_matches) == 1:
        row = last_name_matches[0]
        return {
            'nhl_player_name': row['player_name'],
            'player_id': row['player_id'],
            'match_type': 'last_name',
            'confidence': 0.9
        }
    
    # Strategy 3: Fuzzy match
    best_score = 0
    best_match = None
    for _, row in nhl_names_df.iterrows():
        nhl_norm = normalize_name(row['player_name'])
        score = SequenceMatcher(None, odds_norm, nhl_norm).ratio()
        if score > best_score and score >= 0.8:
            best_score = score
            best_match = row
    
    if best_match is not None:
        return {
            'nhl_player_name': best_match['player_name'],
            'player_id': best_match['player_id'],
            'match_type': 'fuzzy',
            'confidence': best_score
        }
    
    return None

print("Matching function defined.")

## 5. Match Props with Actuals

In [ ]:
# Match all odds props with actuals
if 'odds_df' in dir() and len(odds_df) > 0 and 'actuals_df' in dir() and len(actuals_df) > 0:
    # Take best line per player (e.g., draftkings first)
    book_priority = ['draftkings', 'fanduel', 'betmgm', 'caesars']
    
    def get_book_rank(book):
        return book_priority.index(book) if book in book_priority else 99
    
    odds_df['book_rank'] = odds_df['bookmaker'].apply(get_book_rank)
    
    # Get best over line per player
    over_lines = odds_df[odds_df['bet_type'] == 'Over'].copy()
    over_lines = over_lines.sort_values('book_rank').drop_duplicates(
        subset=['game_date', 'player_name'], 
        keep='first'
    )
    
    print(f"Unique player-game props: {len(over_lines)}")

In [ ]:
# Perform matching
if 'over_lines' in dir() and len(over_lines) > 0:
    matched_results = []
    
    for _, prop in over_lines.iterrows():
        odds_name = prop['player_name']
        
        # Find potential matches from actuals
        match = match_player_names(odds_name, actuals_df)
        
        if match:
            # Get actual SOG
            actual_row = actuals_df[actuals_df['player_id'] == match['player_id']].iloc[0]
            actual_sog = actual_row['shots_on_goal']
            
            result = {
                'game_date': prop['game_date'],
                'odds_player': odds_name,
                'nhl_player': match['nhl_player_name'],
                'match_type': match['match_type'],
                'confidence': match['confidence'],
                'line': prop['line'],
                'odds': prop['odds'],
                'actual_sog': actual_sog,
                'team': actual_row['team_abbrev'],
            }
            
            # Calculate outcome
            if actual_sog > prop['line']:
                result['outcome'] = 'over'
                result['hit'] = True
            elif actual_sog < prop['line']:
                result['outcome'] = 'under'
                result['hit'] = False
            else:
                result['outcome'] = 'push'
                result['hit'] = None
            
            matched_results.append(result)
        else:
            matched_results.append({
                'game_date': prop['game_date'],
                'odds_player': odds_name,
                'nhl_player': None,
                'match_type': 'unmatched',
                'confidence': 0,
                'line': prop['line'],
                'odds': prop['odds'],
                'actual_sog': None,
                'team': None,
                'outcome': None,
                'hit': None,
            })
    
    results_df = pd.DataFrame(matched_results)
    print(f"Total props: {len(results_df)}")
    print("\nMatch rate by type:")
    print(results_df['match_type'].value_counts())

## 6. Analyze Betting Outcomes

In [ ]:
# Filter to matched results only
if 'results_df' in dir() and len(results_df) > 0:
    matched = results_df[results_df['match_type'] != 'unmatched'].copy()
    
    print(f"Matched props: {len(matched)}")
    print("\nOutcome distribution:")
    print(matched['outcome'].value_counts())
    
    # Calculate over hit rate
    non_push = matched[matched['outcome'] != 'push']
    over_hit_rate = non_push['hit'].mean()
    print(f"\nOver hit rate: {over_hit_rate:.1%}")

In [ ]:
# Show sample results
if 'matched' in dir() and len(matched) > 0:
    print("Sample matched props with outcomes:\n")
    display_cols = ['odds_player', 'nhl_player', 'line', 'actual_sog', 'outcome', 'team']
    print(matched[display_cols].head(20).to_string(index=False))

In [ ]:
# Analyze by line value
if 'matched' in dir() and len(matched) > 0:
    matched['line_bucket'] = pd.cut(matched['line'], bins=[0, 2.5, 3.5, 4.5, 10], labels=['1.5-2.5', '2.5-3.5', '3.5-4.5', '4.5+'])
    
    print("Over hit rate by line:\n")
    line_analysis = matched[matched['outcome'] != 'push'].groupby('line_bucket')['hit'].agg(['mean', 'count'])
    line_analysis.columns = ['over_rate', 'n_bets']
    print(line_analysis.to_string())

## 7. Summary & Next Steps

Key findings:
1. Name matching accuracy
2. Over/under hit rates
3. Line efficiency

Next steps:
- Build a proper name mapping table in dbt
- Expand analysis to more dates
- Add ROI calculations based on odds